# Translating OptCom to GAMSpy

I have had a lot of trouble in the implementation of the OptCom model in the micom package. My guess is that the package is just not developed to work with the kind of SBML data that was included in the paper. I do not think that I would be able to make new files, so I am going to do this.

In translating OptCom to python rather than GAMS, I hope to not only make OptCom more accessible in our workflows but also gain a deeper understanding of the mechanisms behind the OptCom system.

In [1]:
# import necessary packages
import pandas as pd
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense, Options, SpecialValues

## Sets

I will need to import the sets as .gdx files. This will use the built in read commands from the container object. I will do all of this later.

In [2]:
cowmunity = Container() # create a container for the model that will hold all of the info, central hub

# create sets for the reactions of each species in the cowmunity
j_mgk = Set(container=cowmunity, name="j_mgk", description="M. Gottschalkii reactions")
j_prm = Set(container=cowmunity, name="j_prm", description="P. ruminicola reactions")
j_rfl = Set(container=cowmunity, name="j_rfl", description="R. flavefaciens reactions")


# create sets for the metabolites for each species
i_mgk = Set(container=cowmunity, name="i_mgk", description="M. Gottschalkii metabolites")
i_prm = Set(container=cowmunity, name="i_prm", description="P. ruminicola metabolites")
i_rfl = Set(container=cowmunity, name="i_rfl", description="R. flavefaciens metabolites")


### Building the Sets as gmx files

I will need to extract the data from each of the models in order to create the sets of reactions and metabolites. Using the rxnnames.txt as a model for what I am going to try to achieve.

In [3]:
import libsbml

def extract_metabolites(xml_file_path, GAMSpy_set):
    # Parse the XML file
    reader = libsbml.SBMLReader()
    document = reader.readSBML(f'model files/{xml_file_path}')
    model = document.getModel()
    if model is None:
        raise ValueError("The SBML model could not be parsed or is empty.")
    
    # Extract names and store them in a list
    met_name = {}
    for i in range(model.getNumSpecies()):
        species = model.getSpecies(i)
        new_entry = {species.id : species.getName()[0:63]}
        met_name.update(new_entry)
    df = pd.DataFrame.from_dict(met_name, orient='index', columns=['name'])
    
    # add the names to the GAMSpy Set
    GAMSpy_set.setRecords(df.reset_index())

    print(f"Successfully extracted {len(GAMSpy_set.records)} metabolite names to {GAMSpy_set.name}")
    print(GAMSpy_set.records.head())

def extract_reactions(xml_file_path, GAMSpy_set):
    # Parse the XML file
    reader = libsbml.SBMLReader()
    document = reader.readSBML(f'model files/{xml_file_path}')
    model = document.getModel()
    if model is None:
        raise ValueError("The SBML model could not be parsed or is empty.")
    
    # Extract names and store them in a list
    rxn_name = {}
    for i in range(model.getNumReactions()):
        reaction = model.getReaction(i)
        new_entry = {reaction.id : reaction.getName()[0:63]}
        rxn_name.update(new_entry)
    df = pd.DataFrame.from_dict(rxn_name, orient='index', columns=['name'])
    
    # add the names to the GAMSpy Set
    GAMSpy_set.setRecords(df.reset_index())

    print(f"Successfully extracted {len(GAMSpy_set.records)} reaction names to {GAMSpy_set.name}")
    print(GAMSpy_set.records.head())




In [4]:
# run the extraction function for each set, these functions will extract the ids of the reactions and metabolites from the SBML files and store them in the GAMSpy Sets
# the actual names of the reactions and metabolites will be stored in the parameters of the Sets, which can be accessed later

extract_reactions('M. gottschalkii.xml', j_mgk)
extract_reactions('P. ruminicola.xml', j_prm)
extract_reactions('R. flavefaciens.xml', j_rfl)

extract_metabolites('M. gottschalkii.xml', i_mgk)
extract_metabolites('P. ruminicola.xml', i_prm)
extract_metabolites('R. flavefaciens.xml', i_rfl)

Successfully extracted 1180 reaction names to j_mgk
                     index              element_text
0            R_acTransport         acetate transport
1   R_arabinose/sodium sym  arabinose/sodium symport
2               R_biomass0                   biomass
3  R_cellbiose_transporter     cellbiose transporter
4  R_cellobiose_hydrolysis     cellobiose hydrolysis
Successfully extracted 1172 reaction names to j_prm
                     index              element_text
0            R_acTransport         acetate transport
1   R_arabinose/sodium sym  arabinose/sodium symport
2               R_biomass0                   biomass
3  R_cellbiose_transporter     cellbiose transporter
4  R_cellobiose_hydrolysis     cellobiose hydrolysis
Successfully extracted 1089 reaction names to j_rfl
                      index            element_text
0       R_acetate_transport       acetate_transport
1                R_biomass0                 biomass
2    R_cellobiose_transport    cellobiose_transport


## Parameters

In [5]:
# define the the upper and lower bounds for the reactions of each species

ub_mgk = Parameter(container=cowmunity, name="ub_mgk", domain=j_mgk, description="Upper bound for M. Gottschalkii reactions")
lb_mgk = Parameter(container=cowmunity, name="lb_mgk", domain=j_mgk, description="Lower bound for M. Gottschalkii reactions")

ub_prm = Parameter(container=cowmunity, name="ub_prm", domain=j_prm, description="Upper bound for P. ruminicola reactions")
lb_prm = Parameter(container=cowmunity, name="lb_prm", domain=j_prm, description="Lower bound for P. ruminicola reactions")

ub_rfl = Parameter(container=cowmunity, name="ub_rfl", domain=j_rfl, description="Upper bound for R. flavefaciens reactions")
lb_rfl = Parameter(container=cowmunity, name="lb_rfl", domain=j_rfl, description="Lower bound for R. flavefaciens reactions")

In [6]:

# define the reaction types for each reaction, whether they are reversible (1) or irreversible (0)

rxntype_mgk = Parameter(container=cowmunity, name="rxntype_mgk", domain=j_mgk, description="Reaction type for M. Gottschalkii reactions")
rxntype_prm = Parameter(container=cowmunity, name="rxntype_prm", domain=j_prm, description="Reaction type for P. ruminicola reactions")
rxntype_rfl = Parameter(container=cowmunity, name="rxntype_rfl", domain=j_rfl, description="Reaction type for R. flavefaciens reactions")

# define the stoichiometric matrix for each species, which describes the relationship between reactions and metabolites
# gives the coefficients for each reaction

S_mgk = Parameter(container=cowmunity, name="S_mgk", domain=[j_mgk, i_mgk], description="Stoichiometric matrix for M. Gottschalkii")
S_prm = Parameter(container=cowmunity, name="S_prm", domain=[j_prm, i_prm], description="Stoichiometric matrix for P. ruminicola")
S_rfl = Parameter(container=cowmunity, name="S_rfl", domain=[j_rfl, i_rfl], description="Stoichiometric matrix for R. flavefaciens")

# define whether each reaction is an exchange reaction

ex_mgk = Parameter(container=cowmunity, name="ex_mgk", domain=j_mgk, description="Exchange for M. Gottschalkii reactions")
ex_prm = Parameter(container=cowmunity, name="ex_prm", domain=j_prm, description="Exchange for P. ruminicola reactions")
ex_rfl = Parameter(container=cowmunity, name="ex_rfl", domain=j_rfl, description="Exchange for R. flavefaciens reactions")


### Extracting the Reaction Types and Matrices from the model files

I will follow a similar approach to what was done above in order to extract the data from the model files.

In [7]:
def extract_rxn_type(xml_file_path, GAMSpy_parameter):
    # Parse the XML file
    reader = libsbml.SBMLReader()
    document = reader.readSBML(f'model files/{xml_file_path}')
    model = document.getModel()
    if model is None:
        raise ValueError("The SBML model could not be parsed or is empty.")
    # Extract names and store them in a list
    type_dict = {}
    for i in range(model.getNumReactions()):
        reaction = model.getReaction(i)
        rxn_type = reaction.getReversible()
        if rxn_type:
            rxn_type = 1
        else:
            rxn_type = 0
        new_entry = {reaction.id : rxn_type}
        type_dict.update(new_entry)
    
    df = pd.DataFrame.from_dict(type_dict, orient='index', columns=['name'])
    
    # add the names to the GAMSpy Set
    GAMSpy_parameter.setRecords(df.reset_index())

    print(f"Successfully extracted {len(GAMSpy_parameter.records)} reaction types to {GAMSpy_parameter.name}")
    print(GAMSpy_parameter.records.head())

def extract_matrix(xml_file_path, GAMSpy_parameter):
    # Parse the XML file
    reader = libsbml.SBMLReader()
    document = reader.readSBML(f'model files/{xml_file_path}')
    model = document.getModel()
    if model is None:
        raise ValueError("The SBML model could not be parsed or is empty.")
    # Extract reactions, metabolites, and their stoichiometry
    df = pd.DataFrame()
    for i in range(model.getNumReactions()):
        reaction = model.getReaction(i)
        for i in range(reaction.getNumReactants()):
            reactant = reaction.getReactant(i)
            metabolite_id = reactant.species
            stoich = reactant.stoichiometry
            new_row_df = pd.DataFrame({'reaction': [reaction.id], 'metabolite': [metabolite_id], 'stoichiometry': [-stoich]})
            df = pd.concat([df, new_row_df], ignore_index=True)
        for i in range(reaction.getNumProducts()):
            product = reaction.getProduct(i)
            metabolite_id = product.species
            stoich = product.stoichiometry
            new_row_df = pd.DataFrame({'reaction': [reaction.id], 'metabolite': [metabolite_id], 'stoichiometry': [stoich]})
            df = pd.concat([df, new_row_df], ignore_index=True)

    df = df.set_index(['reaction', 'metabolite'])

    
    # add the names to the GAMSpy Set
    GAMSpy_parameter.setRecords(df.reset_index())

    print(f"Successfully extracted {len(GAMSpy_parameter.records)} reaction stoichiometries to {GAMSpy_parameter.name}")
    print(GAMSpy_parameter.records.head())

def extract_exchange_type(xml_file_path, GAMSpy_parameter):
    # Parse the XML file
    reader = libsbml.SBMLReader()
    document = reader.readSBML(f'model files/{xml_file_path}')
    model = document.getModel()
    if model is None:
        raise ValueError("The SBML model could not be parsed or is empty.")
    # Extract names and store them in a list
    type_dict = {}
    ex_count = 0
    for i in range(model.getNumReactions()):
        reaction = model.getReaction(i)
        if 'EX_' in reaction.id:
            ex_type = 1
            ex_count += 1
        else:
            ex_type = 0
        new_entry = {reaction.id : ex_type}
        type_dict.update(new_entry)
    
    df = pd.DataFrame.from_dict(type_dict, orient='index', columns=['name'])
    
    # add the names to the GAMSpy Set
    GAMSpy_parameter.setRecords(df.reset_index())

    print(f"Successfully extracted {ex_count} exchange reaction out of {len(GAMSpy_parameter.records)} total reactions to {GAMSpy_parameter.name}")
    print(GAMSpy_parameter.records.head())  

In [8]:
extract_rxn_type('M. gottschalkii.xml', rxntype_mgk)
extract_rxn_type('P. ruminicola.xml', rxntype_prm)
extract_rxn_type('R. flavefaciens.xml', rxntype_rfl)

extract_matrix('M. gottschalkii.xml', S_mgk)
extract_matrix('P. ruminicola.xml', S_prm)
extract_matrix('R. flavefaciens.xml', S_rfl)

extract_exchange_type('M. gottschalkii.xml', ex_mgk)
extract_exchange_type('P. ruminicola.xml', ex_prm)
extract_exchange_type('R. flavefaciens.xml', ex_rfl)

Successfully extracted 1180 reaction types to rxntype_mgk
                     index  value
0            R_acTransport    1.0
1   R_arabinose/sodium sym    0.0
2               R_biomass0    1.0
3  R_cellbiose_transporter    0.0
4  R_cellobiose_hydrolysis    0.0
Successfully extracted 1172 reaction types to rxntype_prm
                     index  value
0            R_acTransport    1.0
1   R_arabinose/sodium sym    0.0
2               R_biomass0    1.0
3  R_cellbiose_transporter    0.0
4  R_cellobiose_hydrolysis    0.0
Successfully extracted 1089 reaction types to rxntype_rfl
                      index  value
0       R_acetate_transport    1.0
1                R_biomass0    0.0
2    R_cellobiose_transport    0.0
3  R_cellodextrin_transport    0.0
4              R_cellulase1    0.0
Successfully extracted 5040 reaction stoichiometries to S_mgk
                 reaction     metabolite  value
0           R_acTransport  M_cpd00029_e0   -1.0
1           R_acTransport  M_cpd00029_c0    1.0
2 

## Scalars

Using scalar definitions to set the default upper and lower bounds of each reaction's flux

In [9]:
# using a scalar parameter to define the maximum flux for each reaction, this is a constant that will be used in the model

Vmax = Parameter(container=cowmunity, name='Vmax', records=1000)

# set the reaction bounds for the irreversible reactions, reaction type 0

ub_mgk[j_mgk].where[rxntype_mgk[j_mgk] == 0] = Vmax
lb_mgk[j_mgk].where[rxntype_mgk[j_mgk] == 0] = SpecialValues.EPS # using SpecialValues.EPS to avoid zero lower bounds, which can cause issues in some solvers

ub_prm[j_prm].where[rxntype_prm[j_prm] == 0] = Vmax
lb_prm[j_prm].where[rxntype_prm[j_prm] == 0] = SpecialValues.EPS

ub_rfl[j_rfl].where[rxntype_rfl[j_rfl] == 0] = Vmax
lb_rfl[j_rfl].where[rxntype_rfl[j_rfl] == 0] = SpecialValues.EPS

# set the reaction bounds for the reversible reactions, reaction type 1

ub_mgk[j_mgk].where[rxntype_mgk[j_mgk] == 1] = Vmax
lb_mgk[j_mgk].where[rxntype_mgk[j_mgk] == 1] = -Vmax

ub_prm[j_prm].where[rxntype_prm[j_prm] == 1] = Vmax
lb_prm[j_prm].where[rxntype_prm[j_prm] == 1] = -Vmax

ub_rfl[j_rfl].where[rxntype_rfl[j_rfl] == 1] = Vmax
lb_rfl[j_rfl].where[rxntype_rfl[j_rfl] == 1] = -Vmax

In [10]:
df = S_prm.records
df.loc[df['reaction'] == 'EX_cpd00027_e0']

,reaction,metabolite,value
119,EX_cpd00027_e0,M_cpd00027_e0,-1.0


## Variables

Setting the names of the pieces of the model that will be changed as it is optimized. This also includes dual variables, which are variables of the dual problem. Every LP problem has its dual problem, an associated problem in reverse. I don't know if I will really need these pieces, but I will do my best to implement them.

For FBA, the dual problem allows one to gain insight into the sensitivity of the optimal solution to changes in the constraints, with these changes called shadow prices.

<u>**Primal Problem (Basic FBA)**</u>

**Maximize or Minimize:** $Z = c^T v$  (Objective Function where c is the vector of the coefficients of the objective "reaction" and v is the flux vector) 

**Subject To:**

- $Sv = 0$ (mass balance, the stoichiometry matrix multiplied by the flux vector must be zero, this is the balance part of FBA)

- $v_{\text{min}} \leq v \leq v_{\text{max}}$ (each flux must be within its upper and lower bounds, constraints set of the problem)

<u>**Dual Problem (Shadow Costs)**</u>

**Minimize**: $Z_D = (v^T_{\text{max}} \mu _{\text{upper}} - v^T_{\text{min}} \mu _{\text{lower}})$ (finds the lowest possible cost to solve the problem, the solution with the least drawbacks, the least better options left on the table)\
(Sum of the upper bound multipled by the shadow cost of changing the upper bound minus the sum of the lower bound multiplied by the cost of changing the lower bound)

**Subject To:**

- $S^{T}\lambda + \mu _{\text{upper}} - \mu _{\text{lower}} = c$\
(the value/costs of all of the consumption/production combined with the costs of the upper and lower limits must add to the total contribution of the reaction to the objective function, this is a balance equation to make sure that all of the costs add up)

- $\mu _{\text{upper}} \geq 0$ (it only makes sense for these costs of changes to the bounds to be positive because it is either a cost or a value depending on whether it is the upper or lower costs)

- $\mu _{\text{lower}} \geq 0$




In [11]:

z_outer = Variable(container=cowmunity, name="z_outer", description="Outer problem objective function")
v_mgk = Variable(container=cowmunity, name="v_mgk", domain=j_mgk, description="Fluxes for M. Gottschalkii reactions")
v_prm = Variable(container=cowmunity, name="v_prm", domain=j_prm, description="Fluxes for P. ruminicola reactions")
v_rfl = Variable(container=cowmunity, name="v_rfl", domain=j_rfl, description="Fluxes for R. flavefaciens reactions")

# Dual variables for the mass balance
lambda_mgk = Variable(container=cowmunity, name="lambda_mgk", domain=i_mgk, description="Dual variables for M. Gottschalkii metabolites")
lambda_prm = Variable(container=cowmunity, name="lambda_prm", domain=i_prm, description="Dual variables for P. ruminicola metabolites")
lambda_rfl = Variable(container=cowmunity, name="lambda_rfl", domain=i_rfl, description="Dual variables for R. flavefaciens metabolites")

# defining the flux bounds for each reaction as the upper and lower bounds defined above
v_mgk.lo[j_mgk] = lb_mgk[j_mgk]
v_mgk.up[j_mgk] = ub_mgk[j_mgk]
v_prm.lo[j_prm] = lb_prm[j_prm]
v_prm.up[j_prm] = ub_prm[j_prm]
v_rfl.lo[j_rfl] = lb_rfl[j_rfl]
v_rfl.up[j_rfl] = ub_rfl[j_rfl]

# medium composition, is there anything else in the medium that i should include
# fixing some of the flux variables to known values, these are the uptakes rates from supplemental data #8
# should I set the O2 intake to zero?
# how do I make sure that they are going to pull from a shared pool? Should these just upper bounds?
# look to figure 1 in the islam paper for more information of where each metabolite goes
v_prm.up['EX_cpd00027_e0'] = 0.794397609 # uptake rate for D-glucose
v_rfl.up['EX_cpd00027_e0'] = 0.794397609
v_rfl.up['EX_cpd00076_e0'] = 0.834726132 # uptake rate for Sucrose
#v_mgk.up['EX_cpd00053_e0'] = 1.597816364 # uptake rate for L-glutamine
v_rfl.up['EX_cpd00107_e0'] = 1.148366562 # uptake rate for L-Leucine
v_prm.up['EX_cpd00224_e0'] = 0.666427627 # uptake rate for L-arabinose
v_rfl.up['EX_cpd00224_e0'] = 0.666427627
v_prm.up['EX_cpd00132_e0'] = 0.400330802 # uptake rate for L-Asparagine
v_rfl.up['EX_cpd00066_e0'] = 0.310015056 # uptake rate for L-Phenylalanine
v_rfl.up['EX_cpd00156_e0'] = 0.385856944 # uptake rate for L-Valine
v_rfl.up['EX_cpd00069_e0'] = 0.218417727 # uptake rate for L-Tyrosine
v_prm.up['EX_cpd00060_e0'] = 0.233929798 # uptake rate for L-Methionine
#v_mgk.up['EX_cpd00322_e0'] = 0.211165444 # uptake rate for L-Isoleucine
v_rfl.up['EX_cpd00322_e0'] = 0.211165444 
#v_mgk.up['EX_cpd00051_e0'] = 0.133660701 # uptake rate for L-Arginine
v_prm.up['EX_cpd00051_e0'] = 0.133660701
v_rfl.up['EX_cpd00051_e0'] = 0.133660701
#v_mgk.up['EX_cpd00039_e0'] = 0.098373937 # uptake rate for L-Lysine
v_prm.up['EX_cpd00039_e0'] = 0.098373937
v_rfl.up['EX_cpd00039_e0'] = 0.098373937
v_prm.up['EX_cpd11746_e0'] = 0.320930655 # uptake rate for Cellulose
v_rfl.up['EX_cpd11746_e0'] = 0.320930655


# Dual variables for bounds
muLB_mgk = Variable(container=cowmunity, name="muLB_mgk", domain=j_mgk, description="Dual variables for MGK LB")
muUB_mgk = Variable(container=cowmunity, name="muUB_mgk", domain=j_mgk, description="Dual variables for MGK UB")
muLB_prm = Variable(container=cowmunity, name="muLB_prm", domain=j_prm, description="Dual variables for PRM LB")
muUB_prm = Variable(container=cowmunity, name="muUB_prm", domain=j_prm, description="Dual variables for PRM UB")
muLB_rfl = Variable(container=cowmunity, name="muLB_rfl", domain=j_rfl, description="Dual variables for RFL LB")
muUB_rfl = Variable(container=cowmunity, name="muUB_rfl", domain=j_rfl, description="Dual variables for RFL UB")

# Set dual variables to positive
muLB_mgk.lo[j_mgk] = 0
muUB_mgk.lo[j_mgk] = 0
muLB_prm.lo[j_prm] = 0
muUB_prm.lo[j_prm] = 0
muLB_rfl.lo[j_rfl] = 0
muUB_rfl.lo[j_rfl] = 0

# transfer uptake positive variables for the shared metabolites

# Redefine transfer variables for MGK without domains
trans_CO2_mgk = Variable(container=cowmunity, name="trans_CO2_mgk", description="maximum transfer of CO2 to M. Gottschalkii")
trans_H2_mgk = Variable(container=cowmunity, name="trans_H2_mgk", description="maximum transfer of H2 to M. Gottschalkii")
trans_formate_mgk = Variable(container=cowmunity, name='trans_formate_mgk', description="maximum transfer of formate to M. Gottschalkii")
trans_acetate_mgk = Variable(container=cowmunity, name="trans_acetate_mgk", description="maximum transfer of acetate to M. Gottschalkii")
trans_d_mannose_mgk = Variable(container=cowmunity, name="trans_dmannose_mgk", description="maximum transfer of D-mannose to M. Gottschalkii")
trans_d_fructose_mgk = Variable(container=cowmunity, name="trans_dfructose_mgk", description="maximum transfer of D-fructose to M. Gottschalkii")
trans_tetrathionate_mgk = Variable(container=cowmunity, name="trans_tetrathionate_mgk", description="maximum transfer of tetrathionate to M. Gottschalkii")
trans_thiosulfate_mgk = Variable(container=cowmunity, name="trans_thiosulfate_mgk", description="maximum transfer of thiosulfate to M. Gottschalkii")

# Redefine transfer variables for PRM without domains
trans_cellulose_prm = Variable(container=cowmunity, name='trans_cellulose_prm', description="maximum transfer of cellulose to P. ruminicola")
trans_d_glucose_prm = Variable(container=cowmunity, name='trans_d_glucose_prm', description="maximum transfer of D-Glucose to P. ruminicola")
trans_acetate_prm = Variable(container=cowmunity, name='trans_acetate_prm', description="maximum transfer of Acetate to P. ruminicola")
trans_l_arabinose_prm = Variable(container=cowmunity, name='trans_l_arabinose_prm', description="maximum transfer of L-Arabinose to P. ruminicola")
trans_biotin_prm = Variable(container=cowmunity, name='trans_biotin_prm', description="maximum transfer of Biotin to P. ruminicola")
trans_thiamin_prm = Variable(container=cowmunity, name='trans_thiamin_prm', description="maximum transfer of Thiamin to P. ruminicola")
trans_octadecenoate_prm = Variable(container=cowmunity, name='trans_octadecenoate_prm', description="maximum transfer of octadecenoate to P. ruminicola")
trans_maltoheptaose_prm = Variable(container=cowmunity, name='trans_maltoheptaose_prm', description="maximum transfer of Maltoheptaose to P. ruminicola")
trans_nitrite_prm = Variable(container=cowmunity, name='trans_nitrite_prm', description="maximum transfer of Nitrite to P. ruminicola")
trans_n_acetyl_d_glucosamine_prm = Variable(container=cowmunity, name='trans_n_acetyl_d_glucosamine_prm', description="maximum transfer of N-Acetyl-D-glucosamine to P. ruminicola")
trans_aminoethanol_prm = Variable(container=cowmunity, name='trans_aminoethanol_prm', description="maximum transfer of Aminoethanol to P. ruminicola")
trans_cobinamide_prm = Variable(container=cowmunity, name='trans_cobinamide_prm', description="maximum transfer of Cobinamide to P. ruminicola")
trans_alanine_prm = Variable(container=cowmunity, name='trans_alanine_prm', description="maximum transfer of Alanine to P. ruminicola")
trans_leucine_prm = Variable(container=cowmunity, name='trans_leucine_prm', description="maximum transfer of Leucine to P. ruminicola")
trans_glycine_prm = Variable(container=cowmunity, name='trans_glycine_prm', description="maximum transfer of Glycine to P. ruminicola")
trans_proline_prm = Variable(container=cowmunity, name='trans_proline_prm', description="maximum transfer of Proline to P. ruminicola")

# Redefine transfer variables for RFL without domains
trans_d_galacturonate_rfl = Variable(container=cowmunity, name='trans_d_galacturonate_rfl', description="maximum transfer of D-Galacturonate to R. flavefaciens")
trans_deoxyadenosine_rfl = Variable(container=cowmunity, name='trans_deoxyadenosine_rfl', description="maximum transfer of Deoxyadenosine to R. flavefaciens")
trans_dephospho_coa_rfl = Variable(container=cowmunity, name='trans_dephospho_coa_rfl', description="maximum transfer of Dephospho-CoA to R. flavefaciens")
trans_cobinamide_rfl = Variable(container=cowmunity, name='trans_cobinamide_rfl', description="maximum transfer of Cobinamide to R. flavefaciens")
trans_alanine_rfl = Variable(container=cowmunity, name='trans_alanine_rfl', description="maximum transfer of Alanine to R. flavefaciens")
trans_leucine_rfl = Variable(container=cowmunity, name='trans_leucine_rfl', description="maximum transfer of Leucine to R. flavefaciens")
trans_glycine_rfl = Variable(container=cowmunity, name='trans_glycine_rfl', description="maximum transfer of Glycine to R. flavefaciens")
trans_proline_rfl = Variable(container=cowmunity, name='trans_proline_rfl', description="maximum transfer of Proline to R. flavefaciens")
trans_aminoethanol_rfl = Variable(container=cowmunity, name='trans_aminoethanol_rfl', description="maximum transfer of Aminoethanol to R. flavefaciens")

# set the transfer variables to positive, these are the maximum transfer rates for each metabolite

trans_CO2_mgk.lo = 0
trans_H2_mgk.lo = 0
trans_formate_mgk.lo = 0
trans_acetate_mgk.lo = 0
trans_d_mannose_mgk.lo = 0
trans_d_fructose_mgk.lo = 0
trans_tetrathionate_mgk.lo = 0
trans_thiosulfate_mgk.lo = 0

trans_cellulose_prm.lo = 0
trans_d_glucose_prm.lo = 0
trans_acetate_prm.lo = 0
trans_l_arabinose_prm.lo = 0
trans_biotin_prm.lo = 0
trans_thiamin_prm.lo = 0
trans_octadecenoate_prm.lo = 0
trans_maltoheptaose_prm.lo = 0
trans_nitrite_prm.lo = 0
trans_n_acetyl_d_glucosamine_prm.lo = 0
trans_aminoethanol_prm.lo = 0
trans_cobinamide_prm.lo = 0
trans_alanine_prm.lo = 0
trans_leucine_prm.lo = 0
trans_glycine_prm.lo = 0
trans_proline_prm.lo = 0

trans_d_galacturonate_rfl.lo = 0
trans_deoxyadenosine_rfl.lo = 0
trans_dephospho_coa_rfl.lo = 0
trans_cobinamide_rfl.lo = 0
trans_alanine_rfl.lo = 0
trans_leucine_rfl.lo = 0
trans_glycine_rfl.lo = 0
trans_proline_rfl.lo = 0
trans_aminoethanol_rfl.lo = 0



# df = v_rfl.records
# df.loc[df['j_rfl'] == 'EX_cpd00027_e0']               - good for checking that the proper constraints are used


## Equations

Adding in the equations that will act as the objectives and constraints on the model

Complex plant carbohydrates and proteins are utilized by R. flavefaciens and P. ruminicola, and the short-chain fatty acids (SCFAs) are absorbed by the rumen epithelium. M. gottschalkii accepts hydrogen, formate, carbon di-oxide from the other two members, and releases methane to the atmosphere

In [12]:
# Objective Functions (Outer Objective)

biomass_objective = Equation(container=cowmunity, name="biomass_objective", description="Objective function for total biomass accumulation")
biomass_objective[...] = z_outer == v_mgk['R_biomass0'] + v_prm['R_biomass0'] + v_rfl['R_biomass0']


#later, add some kind of objective function for minimizing the methane, this is likely the methane transport reaction in mgk or R_rxn03127_c0

# Constraints
# these are the things that are going to make the model into a true community model rather than three separate models

#gas production data goes here!!!!, can be found in the powerpoint from Ratul i think
# Need to find a way to convert mL of CH4 to the proper form for fluxes

# how do I define the equations and whether they are equalities or inequalities?

trans_CO2_mgk_balance = Equation(container=cowmunity, name='trans_CO2_balance', description="Balance for CO2 transfer to M. Gottschalkii")
trans_CO2_mgk_balance[...] = trans_CO2_mgk <= v_prm['EX_cpd00011_e0'] + v_rfl['EX_cpd00011_e0']

trans_H2_mgk_balance = Equation(container=cowmunity, name= 'trans_H2_mgk_balance', description="Balance for H2 transfer to M. Gottschalkii")
trans_H2_mgk_balance[...] = trans_H2_mgk <= v_prm['EX_cpd11640_e0'] + v_rfl['EX_cpd11640_e0']

trans_formate_mgk_balance = Equation(container=cowmunity, name='trans_formate_mgk_balance', description="Balance for formate transfer to M. Gottschalkii")
trans_formate_mgk_balance[...] = trans_formate_mgk <= v_prm['EX_cpd00047_e0'] + v_rfl['EX_cpd00047_e0']

trans_acetate_mgk_balance = Equation(container=cowmunity, name='trans_acetate_mgk_balance', description="Balance for acetate transfer to M. Gottschalkii")
trans_acetate_mgk_balance[...] = trans_acetate_mgk <= v_rfl['EX_cpd00029_e0']

trans_d_mannose_mgk_balance = Equation(container=cowmunity, name='trans_d_mannose_mgk_balance', description="Balance for D-mannose transfer to M. Gottschalkii")
trans_d_mannose_mgk_balance[...] = trans_d_mannose_mgk <= v_rfl['EX_cpd00138_e0']

trans_d_fructose_mgk_balance = Equation(container=cowmunity, name='trans_d_fructose_mgk_balance', description="Balance for D-fructose transfer to M. Gottschalkii")
trans_d_fructose_mgk_balance[...] = trans_d_fructose_mgk <= v_rfl['EX_cpd00082_e0']

trans_tetrathionate_mgk_balance = Equation(container=cowmunity, name='trans_tetrathionate_mgk_balance', description="Balance for tetrathionate transfer to M. Gottschalkii")
trans_tetrathionate_mgk_balance[...] = trans_tetrathionate_mgk <= v_rfl['EX_cpd01414_e0']

trans_thiosulfate_mgk_balance = Equation(container=cowmunity, name='trans_thiosulfate_mgk_balance', description="Balance for thiosulfate transfer to M. Gottschalkii")
trans_thiosulfate_mgk_balance[...] = trans_thiosulfate_mgk <= v_prm['EX_cpd00268_e0']

trans_cellulose_prm_balance = Equation(container=cowmunity, name='trans_cellulose_prm_balance', description="Balance for cellulose transfer to P. ruminicola")
trans_cellulose_prm_balance[...] = trans_cellulose_prm <= v_rfl['EX_cpd11746_e0']

trans_d_glucose_prm_balance = Equation(container=cowmunity, name='trans_d_glucose_prm_balance', description="Balance for D-glucose transfer to P. ruminicola")
trans_d_glucose_prm_balance[...] = trans_d_glucose_prm <= v_rfl['EX_cpd00027_e0']

trans_acetate_prm_balance = Equation(container=cowmunity, name='trans_acetate_prm_balance', description="Balance for acetate transfer to P. ruminicola")
trans_acetate_prm_balance[...] = trans_acetate_prm <= v_rfl['EX_cpd00029_e0']

trans_l_arabinose_prm_balance = Equation(container=cowmunity, name='trans_l_arabinose_prm_balance', description="Balance for L-arabinose transfer to P. ruminicola")
trans_l_arabinose_prm_balance[...] = trans_l_arabinose_prm <= v_rfl['EX_cpd00224_e0']

trans_biotin_prm_balance = Equation(container=cowmunity, name='trans_biotin_prm_balance', description="Balance for biotin transfer to P. ruminicola")
trans_biotin_prm_balance[...] = trans_biotin_prm <= v_rfl['EX_cpd00104_e0']

trans_thiamin_prm_balance = Equation(container=cowmunity, name='trans_thiamin_prm_balance', description="Balance for thiamin transfer to P. ruminicola")
trans_thiamin_prm_balance[...] = trans_thiamin_prm <= v_rfl['EX_cpd00305_e0']

trans_octadecenoate_prm_balance = Equation(container=cowmunity, name='trans_octadecenoate_prm_balance', description="Balance for octadecenoate transfer to P. ruminicola")
trans_octadecenoate_prm_balance[...] = trans_octadecenoate_prm <= v_rfl['EX_cpd15269_e0'] + v_mgk['EX_cpd15269_e0']

trans_maltoheptaose_prm_balance = Equation(container=cowmunity, name='trans_maltoheptaose_prm_balance', description="Balance for maltoheptaose transfer to P. ruminicola")
trans_maltoheptaose_prm_balance[...] = trans_maltoheptaose_prm <= v_rfl['EX_cpd15494_e0']

trans_nitrite_prm_balance = Equation(container=cowmunity, name='trans_nitrite_prm_balance', description="Balance for nitrite transfer to P. ruminicola")
trans_nitrite_prm_balance[...] = trans_nitrite_prm <= v_mgk['EX_cpd00075_e0']

trans_n_acetyl_d_glucosamine_prm_balance = Equation(container=cowmunity, name='trans_n_acetyl_d_glucosamine_prm_balance', description="Balance for N-acetyl-D-glucosamine transfer to P. ruminicola")
trans_n_acetyl_d_glucosamine_prm_balance[...] = trans_n_acetyl_d_glucosamine_prm <= v_mgk['EX_cpd00122_e0']

trans_aminoethanol_prm_balance = Equation(container=cowmunity, name='trans_aminoethanol_prm_balance', description="Balance for aminoethanol transfer to P. ruminicola")
trans_aminoethanol_prm_balance[...] = trans_aminoethanol_prm <= v_mgk['EX_cpd00162_e0']

trans_cobinamide_prm_balance = Equation(container=cowmunity, name='trans_cobinamide_prm_balance', description="Balance for cobinamide transfer to P. ruminicola")
trans_cobinamide_prm_balance[...] = trans_cobinamide_prm <= v_mgk['EX_cpd03422_e0']

trans_alanine_prm_balance = Equation(container=cowmunity, name='trans_alanine_prm_balance', description="Balance for alanine transfer to P. ruminicola")
trans_alanine_prm_balance[...] = trans_alanine_prm <= v_mgk['EX_cpd00035_e0']

trans_leucine_prm_balance = Equation(container=cowmunity, name='trans_leucine_prm_balance', description="Balance for leucine transfer to P. ruminicola")
trans_leucine_prm_balance[...] = trans_leucine_prm <= v_mgk['EX_cpd00107_e0']

trans_glycine_prm_balance = Equation(container=cowmunity, name='trans_glycine_prm_balance', description="Balance for glycine transfer to P. ruminicola")
trans_glycine_prm_balance[...] = trans_glycine_prm <= v_mgk['EX_cpd00033_e0']

trans_proline_prm_balance = Equation(container=cowmunity, name='trans_proline_prm_balance', description="Balance for proline transfer to P. ruminicola")
trans_proline_prm_balance[...] = trans_proline_prm <= v_mgk['EX_cpd00129_e0']

trans_d_galacturonate_rfl_balance = Equation(container=cowmunity, name='trans_d_galacturonate_rfl_balance', description="Balance for D-galacturonate transfer to R. flavefaciens")
trans_d_galacturonate_rfl_balance[...] = trans_d_galacturonate_rfl <= v_prm['EX_cpd00280_e0']

trans_deoxyadenosine_rfl_balance = Equation(container=cowmunity, name='trans_deoxyadenosine_rfl_balance', description="Balance for deoxyadenosine transfer to R. flavefaciens")
trans_deoxyadenosine_rfl_balance[...] = trans_deoxyadenosine_rfl <= v_prm['EX_cpd00438_e0']

trans_dephospho_coa_rfl_balance = Equation(container=cowmunity, name='trans_dephospho_coa_rfl_balance', description="Balance for dephospho-CoA transfer to R. flavefaciens")
trans_dephospho_coa_rfl_balance[...] = trans_dephospho_coa_rfl <= v_mgk['EX_cpd00655_e0']

trans_cobinamide_rfl_balance = Equation(container=cowmunity, name='trans_cobinamide_rfl_balance', description="Balance for cobinamide transfer to R. flavefaciens")
trans_cobinamide_rfl_balance[...] = trans_cobinamide_rfl <= v_mgk['EX_cpd03422_e0']

trans_alanine_rfl_balance = Equation(container=cowmunity, name='trans_alanine_rfl_balance', description="Balance for alanine transfer to R. flavefaciens")
trans_alanine_rfl_balance[...] = trans_alanine_rfl <= v_mgk['EX_cpd00035_e0']

trans_leucine_rfl_balance = Equation(container=cowmunity, name='trans_leucine_rfl_balance', description="Balance for leucine transfer to R. flavefaciens")
trans_leucine_rfl_balance[...] = trans_leucine_rfl <= v_mgk['EX_cpd00107_e0']

trans_glycine_rfl_balance = Equation(container=cowmunity, name='trans_glycine_rfl_balance', description="Balance for glycine transfer to R. flavefaciens")
trans_glycine_rfl_balance[...] = trans_glycine_rfl <= v_mgk['EX_cpd00033_e0']

trans_proline_rfl_balance = Equation(container=cowmunity, name='trans_proline_rfl_balance', description="Balance for proline transfer to R. flavefaciens")
trans_proline_rfl_balance[...] = trans_proline_rfl <= v_mgk['EX_cpd00129_e0']

trans_aminoethanol_rfl_balance = Equation(container=cowmunity, name='trans_aminoethanol_rfl_balance', description="Balance for aminoethanol transfer to R. flavefaciens")
trans_aminoethanol_rfl_balance[...] = trans_aminoethanol_rfl <= v_mgk['EX_cpd00162_e0']

overall_cobinamide_balance = Equation(container=cowmunity, name='overall_cobinamide_balance', description='Balance for cobinamide transfer for both prm and rfl')
overall_cobinamide_balance[...] = trans_cobinamide_prm + trans_cobinamide_rfl <= v_mgk['EX_cpd03422_e0']

overall_alanine_balance = Equation(container=cowmunity, name='overall_alanine_balance', description='Balance for alanine transfer for both prm and rfl')
overall_alanine_balance[...] = trans_alanine_prm + trans_alanine_rfl <= v_mgk['EX_cpd00035_e0']

overall_leucine_balance = Equation(container=cowmunity, name='overall_leucine_balance', description='Balance for leucine transfer for both prm and rfl')
overall_leucine_balance[...] = trans_leucine_prm + trans_leucine_rfl <= v_mgk['EX_cpd00107_e0']

overall_glycine_balance = Equation(container=cowmunity, name='overall_glycine_balance', description='Balance for glycine transfer for both prm and rfl')
overall_glycine_balance[...] = trans_glycine_prm + trans_glycine_rfl <= v_mgk['EX_cpd00033_e0']

overall_proline_balance = Equation(container=cowmunity, name='overall_proline_balance', description='Balance for proline transfer for both prm and rfl')
overall_proline_balance[...] = trans_proline_prm + trans_proline_rfl <= v_mgk['EX_cpd00129_e0']

overall_aminoethanol_balance = Equation(container=cowmunity, name='overall_aminoethanol_balance', description='Balance for aminoethanol transfer for both prm and rfl')
overall_aminoethanol_balance[...] = trans_aminoethanol_prm + trans_aminoethanol_rfl <= v_mgk['EX_cpd00162_e0']

# adding the constraints to each model

constraint_CO2_mgk = Equation(container=cowmunity, name="constraint_CO2_mgk", description="constraint for the uptake of CO2 to M. Gottschalkii")
constraint_CO2_mgk[...] = trans_CO2_mgk == v_mgk['EX_cpd00011_e0']

constraint_H2_mgk = Equation(container=cowmunity, name="constraint_H2_mgk", description="constraint for the uptake of H2 to M. Gottschalkii")
constraint_H2_mgk[...] = trans_H2_mgk == v_mgk['EX_cpd11640_e0']

constraint_formate_mgk = Equation(container=cowmunity, name='constraint_formate_mgk', description="constraint for the uptake of formate to M. Gottschalkii")
constraint_formate_mgk[...] = trans_formate_mgk == v_mgk['EX_cpd00047_e0']

constraint_acetate_mgk = Equation(container=cowmunity, name="constraint_acetate_mgk", description="constraint for the uptake of acetate to M. Gottschalkii")
constraint_acetate_mgk[...] = trans_acetate_mgk == v_mgk['EX_cpd00029_e0']

constraint_d_mannose_mgk = Equation(container=cowmunity, name="constraint_dmannose_mgk", description="constraint for the uptake of D-mannose to M. Gottschalkii")
constraint_d_mannose_mgk[...] = trans_d_mannose_mgk == v_mgk['EX_cpd00138_e0']

constraint_d_fructose_mgk = Equation(container=cowmunity, name="constraint_dfructose_mgk", description="constraint for the uptake of D-fructose to M. Gottschalkii")
constraint_d_fructose_mgk[...] = trans_d_fructose_mgk == v_mgk['EX_cpd00082_e0']

constraint_tetrathionate_mgk = Equation(container=cowmunity, name="constraint_tetrathionate_mgk", description="constraint for the uptake of tetrathionate to M. Gottschalkii")
constraint_tetrathionate_mgk[...] = trans_tetrathionate_mgk == v_mgk['EX_cpd01414_e0']

constraint_thiosulfate_mgk = Equation(container=cowmunity, name="constraint_thiosulfate_mgk", description="constraint for the uptake of thiosulfate to M. Gottschalkii")
constraint_thiosulfate_mgk[...] = trans_thiosulfate_mgk == v_mgk['EX_cpd00268_e0']

constraint_cellulose_prm = Equation(container=cowmunity, name='constraint_cellulose_prm', description="constraint for the uptake of cellulose to P. ruminicola")
constraint_cellulose_prm[...] = trans_cellulose_prm == v_prm['EX_cpd11746_e0']

constraint_d_glucose_prm = Equation(container=cowmunity, name='constraint_d_glucose_prm', description="constraint for the uptake of D-Glucose to P. ruminicola")
constraint_d_glucose_prm[...] = trans_d_glucose_prm == v_prm['EX_cpd00027_e0']

constraint_acetate_prm = Equation(container=cowmunity, name='constraint_acetate_prm', description="constraint for the uptake of Acetate to P. ruminicola")
constraint_acetate_prm[...] = trans_acetate_prm == v_prm['EX_cpd00029_e0']

constraint_l_arabinose_prm = Equation(container=cowmunity, name='constraint_l_arabinose_prm', description="constraint for the uptake of L-Arabinose to P. ruminicola")
constraint_l_arabinose_prm[...] = trans_l_arabinose_prm == v_prm['EX_cpd00224_e0']

constraint_biotin_prm = Equation(container=cowmunity, name='constraint_biotin_prm', description="constraint for the uptake of Biotin to P. ruminicola")
constraint_biotin_prm[...] = trans_biotin_prm == v_prm['EX_cpd00104_e0']

constraint_thiamin_prm = Equation(container=cowmunity, name='constraint_thiamin_prm', description="constraint for the uptake of Thiamin to P. ruminicola")
constraint_thiamin_prm[...] = trans_thiamin_prm == v_prm['EX_cpd00305_e0']

constraint_octadecenoate_prm = Equation(container=cowmunity, name='constraint_octadecenoate_prm', description="constraint for the uptake of octadecenoate to P. ruminicola")
constraint_octadecenoate_prm[...] = trans_octadecenoate_prm == v_prm['EX_cpd15269_e0']

constraint_maltoheptaose_prm = Equation(container=cowmunity, name='constraint_maltoheptaose_prm', description="constraint for the uptake of Maltoheptaose to P. ruminicola")
constraint_maltoheptaose_prm[...] = trans_maltoheptaose_prm == v_prm['EX_cpd15494_e0']

constraint_nitrite_prm = Equation(container=cowmunity, name='constraint_nitrite_prm', description="constraint for the uptake of Nitrite to P. ruminicola")
constraint_nitrite_prm[...] = trans_nitrite_prm == v_prm['EX_cpd00075_e0']

constraint_n_acetyl_d_glucosamine_prm = Equation(container=cowmunity, name='constraint_n_acetyl_d_glucosamine_prm', description="constraint for the uptake of N-Acetyl-D-glucosamine to P. ruminicola")
constraint_n_acetyl_d_glucosamine_prm[...] = trans_n_acetyl_d_glucosamine_prm == v_prm['EX_cpd00122_e0']

constraint_aminoethanol_prm = Equation(container=cowmunity, name='constraint_aminoethanol_prm', description="constraint for the uptake of Aminoethanol to P. ruminicola")
constraint_aminoethanol_prm[...] = trans_aminoethanol_prm == v_prm['EX_cpd00162_e0']

constraint_cobinamide_prm = Equation(container=cowmunity, name='constraint_cobinamide_prm', description="constraint for the uptake of Cobinamide to P. ruminicola")
constraint_cobinamide_prm[...] = trans_cobinamide_prm == v_prm['EX_cpd03422_e0']

constraint_alanine_prm = Equation(container=cowmunity, name='constraint_alanine_prm', description="constraint for the uptake of Alanine to P. ruminicola")
constraint_alanine_prm[...] = trans_alanine_prm == v_prm['EX_cpd00035_e0']

constraint_leucine_prm = Equation(container=cowmunity, name='constraint_leucine_prm', description="constraint for the uptake of Leucine to P. ruminicola")
constraint_leucine_prm[...] = trans_leucine_prm == v_prm['EX_cpd00107_e0']

constraint_glycine_prm = Equation(container=cowmunity, name='constraint_glycine_prm', description="constraint for the uptake of Glycine to P. ruminicola")
constraint_glycine_prm[...] = trans_glycine_prm == v_prm['EX_cpd00033_e0']

constraint_proline_prm = Equation(container=cowmunity, name='constraint_proline_prm', description="constraint for the uptake of Proline to P. ruminicola")
constraint_proline_prm[...] = trans_proline_prm == v_prm['EX_cpd00129_e0']

constraint_d_galacturonate_rfl = Equation(container=cowmunity, name='constraint_d_galacturonate_rfl', description="constraint for the uptake of D-Galacturonate to R. flavefaciens")
constraint_d_galacturonate_rfl[...] = trans_d_galacturonate_rfl == v_rfl['EX_cpd00280_e0']

constraint_deoxyadenosine_rfl = Equation(container=cowmunity, name='constraint_deoxyadenosine_rfl', description="constraint for the uptake of Deoxyadenosine to R. flavefaciens")
constraint_deoxyadenosine_rfl[...] = trans_deoxyadenosine_rfl == v_rfl['EX_cpd00438_e0']

constraint_dephospho_coa_rfl = Equation(container=cowmunity, name='constraint_dephospho_coa_rfl', description="constraint for the uptake of Dephospho-CoA to R. flavefaciens")
constraint_dephospho_coa_rfl[...] = trans_dephospho_coa_rfl == v_rfl['EX_cpd00655_e0']

constraint_cobinamide_rfl = Equation(container=cowmunity, name='constraint_cobinamide_rfl', description="constraint for the uptake of Cobinamide to R. flavefaciens")
constraint_cobinamide_rfl[...] = trans_cobinamide_rfl == v_rfl['EX_cpd03422_e0']

constraint_alanine_rfl = Equation(container=cowmunity, name='constraint_alanine_rfl', description="constraint for the uptake of Alanine to R. flavefaciens")
constraint_alanine_rfl[...] = trans_alanine_rfl == v_rfl['EX_cpd00035_e0']

constraint_leucine_rfl = Equation(container=cowmunity, name='constraint_leucine_rfl', description="constraint for the uptake of Leucine to R. flavefaciens")
constraint_leucine_rfl[...] = trans_leucine_rfl == v_rfl['EX_cpd00107_e0']

constraint_glycine_rfl = Equation(container=cowmunity, name='constraint_glycine_rfl', description="constraint for the uptake of Glycine to R. flavefaciens")
constraint_glycine_rfl[...] = trans_glycine_rfl == v_rfl['EX_cpd00033_e0']

constraint_proline_rfl = Equation(container=cowmunity, name='constraint_proline_rfl', description="constraint for the uptake of Proline to R. flavefaciens")
constraint_proline_rfl[...] = trans_proline_rfl == v_rfl['EX_cpd00129_e0']

constraint_aminoethanol_rfl = Equation(container=cowmunity, name='constraint_aminoethanol_rfl', description="constraint for the uptake of Aminoethanol to R. flavefaciens")
constraint_aminoethanol_rfl[...] = trans_aminoethanol_rfl == v_rfl['EX_cpd00162_e0']

Verifying the reaction ids for the constraints, make sure that everything line's up

In [13]:
reaction_ids_mgk = [
    'EX_cpd00011_e0',  # CO2
    'EX_cpd11640_e0',  # H2
    'EX_cpd00047_e0',  # formate
    'EX_cpd00029_e0',  # acetate
    'EX_cpd00138_e0',  # D-mannose
    'EX_cpd00082_e0',  # D-fructose
    'EX_cpd01414_e0',  # tetrathionate
    'EX_cpd00268_e0',  # thiosulfate
]

reaction_ids_prm = [
    'EX_cpd11746_e0',  # cellulose
    'EX_cpd00027_e0',  # D-glucose
    'EX_cpd00029_e0',  # acetate
    'EX_cpd00224_e0',  # L-arabinose
    'EX_cpd00104_e0',  # biotin
    'EX_cpd00305_e0',  # thiamin
    'EX_cpd15269_e0',  # octadecenoate
    'EX_cpd15494_e0',  # maltoheptaose
    'EX_cpd00075_e0',  # nitrite
    'EX_cpd00122_e0',  # N-acetyl-D-glucosamine
    'EX_cpd00162_e0',  # aminoethanol
    'EX_cpd03422_e0',  # cobinamide
    'EX_cpd00035_e0',  # alanine
    'EX_cpd00107_e0',  # leucine
    'EX_cpd00033_e0',  # glycine
    'EX_cpd00129_e0',  # proline
]

reaction_ids_rfl = [
    'EX_cpd00280_e0',  # D-galacturonate
    'EX_cpd00438_e0',  # deoxyadenosine
    'EX_cpd00655_e0',  # dephospho-CoA
    'EX_cpd00162_e0',  # aminoethanol
    'EX_cpd03422_e0',  # cobinamide
    'EX_cpd00035_e0',  # alanine
    'EX_cpd00107_e0',  # leucine
    'EX_cpd00033_e0',  # glycine
    'EX_cpd00129_e0',  # proline
]


def list_reaction_names(id_set, i_set, file_name):
    reader = libsbml.SBMLReader()
    document = reader.readSBML(f'model files/{file_name}')
    model = document.getModel()

    metabolite_list = []
    for item in id_set:
        reaction = model.getReaction(f'{item}')
        reactant = reaction.getReactant(f'M_{item[3:11]}_e0')
        metabolite_list.append(reactant.getSpecies())

    df = i_set.records
    for item in metabolite_list:
        print(df.loc[df['index'] == item])

# list_reaction_names(reaction_ids_mgk, i_mgk, 'M. gottschalkii.xml')
# list_reaction_names(reaction_ids_prm, i_prm, 'P. ruminicola.xml')
# list_reaction_names(reaction_ids_rfl, i_rfl, 'R. flavefaciens.xml')

In [59]:
# example of how to use df's and .loc

df = i_mgk.records
df
df.loc[df['index'] == 'EX_cpd00011_e0']

,index,element_text


In [15]:
# mass balance equations

mgk_mass_balance = Equation(container=cowmunity, name='mgk_mass_balance', domain=i_mgk, description='overall mass balance for mgk')
mgk_mass_balance[i_mgk] = Sum(j_mgk, S_mgk[j_mgk, i_mgk] * v_mgk[j_mgk]) == 0

prm_mass_balance = Equation(container=cowmunity, name='prm_mass_balance', domain=i_prm, description='overall mass balance for prm')
prm_mass_balance[i_prm] = Sum(j_prm, S_prm[j_prm, i_prm] * v_prm[j_prm]) == 0

rfl_mass_balance = Equation(container=cowmunity, name='rfl_mass_balance', domain=i_rfl, description='overall mass balance for rfl')
rfl_mass_balance[i_rfl] = Sum(j_rfl, S_rfl[j_rfl, i_rfl] * v_rfl[j_rfl]) == 0

# dual problem constraint equations

mgk_dual_constraint = Equation(container=cowmunity, name="mgk_dual_constraint", domain=j_mgk, description="Dual constraint for MGK")
mgk_dual_constraint[j_mgk] = Sum(i_mgk, lambda_mgk[i_mgk] * S_mgk[j_mgk, i_mgk]) + \
                             muUB_mgk[j_mgk] - muLB_mgk[j_mgk] == 0

prm_dual_constraint = Equation(container=cowmunity, name="prm_dual_constraint", domain=j_prm, description="Dual constraint for PRM")
prm_dual_constraint[j_prm] = Sum(i_prm, lambda_prm[i_prm] * S_prm[j_prm, i_prm]) + \
                             muUB_prm[j_prm] - muLB_prm[j_prm] == 0

rfl_dual_constraint = Equation(container=cowmunity, name="rfl_dual_constraint", domain=j_rfl, description="Dual constraint for RFL")
rfl_dual_constraint[j_rfl] = Sum(i_rfl, lambda_rfl[i_rfl] * S_rfl[j_rfl, i_rfl]) + \
                             muUB_rfl[j_rfl] - muLB_rfl[j_rfl] == 0

# special dual problem constraints for other species, could add some for methane later



# It's solver time babyyyyyy!!!

The solver function for the model. This function will build the model based on everything that we have defined above and set the options for the solve as well.

In [16]:
from gamspy import SolveStatus

def solve(solver_name='IPOPT'):
        global model
        # Create model
        model = Model(container=cowmunity, name="COWMUNITY", equations=cowmunity.getEquations(), 
                     problem="NLP", sense=Sense.MAX, objective=z_outer)
        
        # Set solver options
        options = Options(nlp=solver_name, equation_listing_limit=10, variable_listing_limit=10)

        
        # Solve the model
        print(f"Solving with {solver_name}...")
        model.solve(options=options)
        
        # check solution status
        status = model.solve_status
        if status == SolveStatus.NormalCompletion:
            print("Optimal solution found!")
            print(model.objective_value)
        else:
            print(f"Solver terminated with status: {status}")

solve()


Solving with IPOPT...
Optimal solution found!
8.507856965391326e-08


In [78]:
def biomass_pieces(file_name, v_set, j_set):
    reader = libsbml.SBMLReader()
    document = reader.readSBML(f'model files/{file_name}')
    model = document.getModel()

    biomass_reactants = []
    biomass_reaction = model.getReaction('R_biomass0')
    for i in range(biomass_reaction.getNumReactants()):
        reactant = biomass_reaction.getReactant(i)
        biomass_reactants.append(reactant.getSpecies())

    precursor_reactions = {}
    j = 0
    for item in biomass_reactants:
        relevant_reactions = []
        for i in range(model.getNumReactions()):
            reaction = model.getReaction(i)
            product = reaction.getProduct(f'{item}')
            if product is None:
                continue
            else:
                relevant_reactions.append(reaction.id)
        new_entry = {item : relevant_reactions}
        precursor_reactions.update(new_entry)

    print(precursor_reactions)

    for key in precursor_reactions:
        relevant_reactions = precursor_reactions[key]
        species = biomass_reaction.getReactant(f'{key}').getSpecies()
        name = i_mgk.records.loc[df['index'] == species, 'element_text'].iloc[0]
        print(f'*** {name} FLUXES ***')
        for item in relevant_reactions:
            flux = v_set.records.loc[v_set.records[j_set] == f'{item}', 'level'].iloc[0]
            print(f'{item} : {flux}')


biomass_pieces('M. gottschalkii.xml', v_mgk, 'j_mgk')


{'M_cpd00001_c0': ['R_rxn00215_c0', 'R_rxn00459_c0', 'R_rxn00474_c0', 'R_rxn00623_c0', 'R_rxn00642_c0', 'R_rxn00726_c0', 'R_rxn00776_c0', 'R_rxn00799_c0', 'R_rxn00898_c0', 'R_rxn01000_c0', 'R_rxn01023_c0', 'R_rxn01100_c0', 'R_rxn01157_c0', 'R_rxn01633_c0', 'R_rxn01635_c0', 'R_rxn01639_c0', 'R_rxn01750_c0', 'R_rxn01964_c0', 'R_rxn01997_c0', 'R_rxn02200_c0', 'R_rxn02213_c0', 'R_rxn02264_c0', 'R_rxn02304_c0', 'R_rxn02374_c0', 'R_rxn02473_c0', 'R_rxn02507_c0', 'R_rxn02789_c0', 'R_rxn02811_c0', 'R_rxn02821_c0', 'R_rxn02832_c0', 'R_rxn02916_c0', 'R_rxn02936_c0', 'R_rxn02988_c0', 'R_rxn03012_c0', 'R_rxn03061_c0', 'R_rxn03080_c0', 'R_rxn03174_c0', 'R_rxn03437_c0', 'R_rxn03465_c0', 'R_rxn03885_c0', 'R_rxn03887_c0', 'R_rxn04113_c0', 'R_rxn04139_c0', 'R_rxn04996_c0', 'R_rxn05024_c0', 'R_rxn05144_c0', 'R_rxn05232_c0', 'R_rxn05234_c0', 'R_rxn05235_c0', 'R_rxn05236_c0', 'R_rxn05293_c0', 'R_rxn05361_c0', 'R_rxn05365_c0', 'R_rxn05369_c0', 'R_rxn05373_c0', 'R_rxn05377_c0', 'R_rxn05381_c0', 'R_rxn05386_

IndexError: single positional indexer is out-of-bounds

In [ ]:
species = 'M_cpd00001_c0'
print(i_mgk.records.loc[df['index'] == species, 'element_text'].iloc[0])

13    H2O[c0]
Name: element_text, dtype: object


In [41]:
v_mgk.records.loc[v_mgk.records['j_mgk'] == 'R_rxn05343_c0', 'level'].iloc[0]

np.float64(-3.358725525595135e-13)

In [19]:
def _extract_results():
        """Extract and store results"""
        results = {
            'objective_value': z_outer.records['level'].iloc[0],
            'mgk_biomass': v_mgk.records.loc[v_mgk.records['jsyn'] == 'syn16_bm', 'level'].iloc[0],
            'prm_biomass': v_prm.records.loc[v_prm.records['jfap'] == 'fap25_bm', 'level'].iloc[0],
            'rfl_biomass': v_rfl.records.loc[v_rfl.records['jsrb'] == 'srb22_bm', 'level'].iloc[0],
            'syn21_flux': self.vsyn.records.loc[self.vsyn.records['jsyn'] == 'syn21', 'level'].iloc[0],
            'syn24_flux': self.vsyn.records.loc[self.vsyn.records['jsyn'] == 'syn24', 'level'].iloc[0],
            'O2_CO2_ratio': self.syn21_syn24_ratio
        }

In [49]:
v_mgk.records
j_set = 'j_mgk'

print(v_mgk.records.loc[v_mgk.records[j_set] == 'R_biomass0', 'level'].iloc[0])

2.0952126971027624e-08
